In [1]:
import pandas as pd
from datetime import datetime

df = pd.read_csv('turnstile_200411.txt')

In [2]:
#Remove any leading / trailing spaces from the header
df.rename(columns=lambda x: x.strip(), inplace=True)

#Cast the date from object to datetime and add a new column for the Day
df['DATE'] = pd.to_datetime(df['DATE'])
df['DAY'] = df['DATE'].dt.strftime('%a')

In [3]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205981 entries, 0 to 205980
Data columns (total 12 columns):
C/A         205981 non-null object
UNIT        205981 non-null object
SCP         205981 non-null object
STATION     205981 non-null object
LINENAME    205981 non-null object
DIVISION    205981 non-null object
DATE        205981 non-null datetime64[ns]
TIME        205981 non-null object
DESC        205981 non-null object
ENTRIES     205981 non-null int64
EXITS       205981 non-null int64
DAY         205981 non-null object
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 18.9+ MB


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DAY
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-04,00:00:00,REGULAR,7413507,2516927,Sat
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-04,04:00:00,REGULAR,7413507,2516927,Sat
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-04,08:00:00,REGULAR,7413511,2516935,Sat
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-04,12:00:00,REGULAR,7413521,2516944,Sat
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-04,16:00:00,REGULAR,7413541,2516953,Sat


In [4]:
# Data Visualization

#df.TIME.unique()
#df.SCP.value_counts()
#df.STATION.value_counts()
#df.UNIT.value_counts()

In [5]:
# Obtain the hour portion from the timestamp and store as a new integer column
df['Hour'] = df.TIME.str.slice(0, 2).astype(int)

# 0600 - 0800 start times considered Morning
# 0900 - 1500 start times considered Afternoon
# 1600 onwards considered Evening
df['TimeofDay'] = df['Hour'].apply(lambda x: 'Morning' if (x>=6 and x<=8) else ('Afternoon' if (x>=9 and x<=15) else 'Evening'))


In [6]:
df.Hour.unique()
#df.head(50)

array([ 0,  4,  8, 12, 16, 20,  2,  6, 10, 14, 18, 22,  1,  5,  9, 13, 17,
       21,  7,  3, 11, 15, 19, 23], dtype=int64)

In [7]:
#df['Hourval'].value_counts(sort=False)

In [8]:

#df['ActualEntry'] = df['ENTRIES'] - df['ENTRIES'].shift(1)
#df['ActualExit'] = df['EXITS'] - df['EXITS'].shift(1)

#df = df.sort_values(by=['SCP','DATE'])

# New Columns to calculate the actual entries and exits from the cumulative numbers
# Logic included to set to zero if the SCP is different from the previous entry
# OR if the cumulative amount for the current row is less than the previous row's
df['ActualEntry'] = df['ENTRIES'].diff()
#df.loc[df.SCP != df.SCP.shift(), 'ActualEntry1'] = 0
df.loc[(df['SCP'] != df['SCP'].shift()) | (df['ENTRIES'] < df['ENTRIES'].shift()), 'ActualEntry'] = 0


df['ActualExit'] = df['EXITS'].diff()
#df.loc[df.SCP != df.SCP.shift(), 'ActualEntry1'] = 0
df.loc[(df['SCP'] != df['SCP'].shift()) | (df['EXITS'] < df['EXITS'].shift()), 'ActualExit'] = 0
df.head(50)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DAY,Hour,TimeofDay,ActualEntry,ActualExit
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-04,00:00:00,REGULAR,7413507,2516927,Sat,0,Evening,0.0,0.0
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-04,04:00:00,REGULAR,7413507,2516927,Sat,4,Evening,0.0,0.0
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-04,08:00:00,REGULAR,7413511,2516935,Sat,8,Morning,4.0,8.0
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-04,12:00:00,REGULAR,7413521,2516944,Sat,12,Afternoon,10.0,9.0
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-04,16:00:00,REGULAR,7413541,2516953,Sat,16,Evening,20.0,9.0
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-04,20:00:00,REGULAR,7413567,2516958,Sat,20,Evening,26.0,5.0
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-05,00:00:00,REGULAR,7413584,2516966,Sun,0,Evening,17.0,8.0
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-05,04:00:00,REGULAR,7413585,2516968,Sun,4,Evening,1.0,2.0
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-05,08:00:00,REGULAR,7413591,2516973,Sun,8,Morning,6.0,5.0
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-05,12:00:00,REGULAR,7413597,2516983,Sun,12,Afternoon,6.0,10.0


In [9]:
# Filter data to exclude hours that are too early or late
# Range included is effectively from 6am to 10pm (since 6pm timing includes the 4 hours after) 
df = df[(df.Hour>=6) & (df.Hour<=19)]
df.head(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DAY,Hour,TimeofDay,ActualEntry,ActualExit
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-04,08:00:00,REGULAR,7413511,2516935,Sat,8,Morning,4.0,8.0
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-04,12:00:00,REGULAR,7413521,2516944,Sat,12,Afternoon,10.0,9.0
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-04,16:00:00,REGULAR,7413541,2516953,Sat,16,Evening,20.0,9.0
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-05,08:00:00,REGULAR,7413591,2516973,Sun,8,Morning,6.0,5.0
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-05,12:00:00,REGULAR,7413597,2516983,Sun,12,Afternoon,6.0,10.0
10,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-05,16:00:00,REGULAR,7413608,2516997,Sun,16,Evening,11.0,14.0
14,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-06,08:00:00,REGULAR,7413636,2517017,Mon,8,Morning,1.0,10.0
15,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-06,12:00:00,REGULAR,7413648,2517033,Mon,12,Afternoon,12.0,16.0
16,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-06,16:00:00,REGULAR,7413678,2517045,Mon,16,Evening,30.0,12.0
20,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-07,08:00:00,REGULAR,7413743,2517058,Tue,8,Morning,3.0,9.0


In [10]:
# Check for negatives (which should not exist)
df[(df.ActualEntry < 0) | (df.ActualExit < 0)]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DAY,Hour,TimeofDay,ActualEntry,ActualExit


In [11]:
# Check for zeroes (get a sense of the values)
df[(df.ActualEntry == 0) | (df.ActualExit == 0)]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DAY,Hour,TimeofDay,ActualEntry,ActualExit
98,A002,R051,02-03-00,59 ST,NQR456W,BMT,2020-04-06,08:00:00,REGULAR,1376450,5179880,Mon,8,Morning,0.0,34.0
104,A002,R051,02-03-00,59 ST,NQR456W,BMT,2020-04-07,08:00:00,REGULAR,1376468,5180021,Tue,8,Morning,0.0,45.0
116,A002,R051,02-03-00,59 ST,NQR456W,BMT,2020-04-09,08:00:00,REGULAR,1376507,5180241,Thu,8,Morning,0.0,20.0
338,A002,R051,02-03-06,59 ST,NQR456W,BMT,2020-04-04,08:00:00,REGULAR,9340403,507309,Sat,8,Morning,9.0,0.0
350,A002,R051,02-03-06,59 ST,NQR456W,BMT,2020-04-06,08:00:00,REGULAR,9340542,507317,Mon,8,Morning,10.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205972,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2020-04-09,13:00:00,REGULAR,5554,514,Thu,13,Afternoon,0.0,0.0
205973,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2020-04-09,17:00:00,REGULAR,5554,514,Thu,17,Evening,0.0,0.0
205977,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2020-04-10,09:00:00,REGULAR,5554,514,Fri,9,Afternoon,0.0,0.0
205978,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2020-04-10,13:00:00,REGULAR,5554,514,Fri,13,Afternoon,0.0,0.0


In [12]:
# New column to calculate the Entry and Exits per 4 hour slot
df['IntervalTotal'] = df['ActualEntry'] + df['ActualExit']

In [13]:
# New column to calculate the daily totals for each station.
df['DailyTotal'] = df.groupby(['STATION','DATE'])['IntervalTotal'].transform('sum')

In [14]:
# New column to calculate the daily totals for each station.
df['WeeklyTotal'] = df.groupby(['STATION'])['IntervalTotal'].transform('sum')

In [15]:
# New column to calculate the daily totals for each station.
df['TimeofDayTotal'] = df.groupby(['STATION','TimeofDay'])['IntervalTotal'].transform('sum')

In [16]:
df = df[df.STATION != '46 ST']

In [19]:
# Viewing a subset of the data sorted by DailyTotal in descending order and only the unique values
dfDaily = df[['STATION','DATE','DAY','DailyTotal']].sort_values(by='DailyTotal', ascending=False).drop_duplicates()
dfDaily.head(50)

,STATION,DATE,DAY,DailyTotal
47266,125 ST,2020-04-06,Mon,15981.0
167027,125 ST,2020-04-08,Wed,15703.0
47018,125 ST,2020-04-07,Tue,15701.0
171364,125 ST,2020-04-10,Fri,15650.0
171483,125 ST,2020-04-09,Thu,14888.0
54421,34 ST-PENN STA,2020-04-06,Mon,11962.0
137666,34 ST-PENN STA,2020-04-08,Wed,11530.0
55358,34 ST-PENN STA,2020-04-07,Tue,11474.0
53892,34 ST-PENN STA,2020-04-09,Thu,11335.0
166921,125 ST,2020-04-04,Sat,11237.0


In [ ]:
# Viewing a subset of the data sorted by DailyTotal in descending order and only the unique values
dfTOD = df[['STATION','TimeofDay','TimeofDayTotal']].sort_values(by='TimeofDayTotal', ascending=False).drop_duplicates()
dfTOD

In [ ]:
dfTOD.nlargest(50, 'TimeofDayTotal')

In [ ]:
# Viewing a subset of the data sorted by DailyTotal in descending order and only the unique values
df[['STATION','WeeklyTotal']].sort_values(by='WeeklyTotal', ascending=False).drop_duplicates().nlargest(10, 'WeeklyTotal')

In [ ]:
dfDaily[(dfDaily.STATION == "125 ST")].DailyTotal

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.distplot(dfDaily['DailyTotal'], kde=False,bins=8,norm_hist=True);
sns.barplot(x = 'DAY',y='DailyTotal',data=dfDaily);

plt.figure(figsize=[10,8])

y1 = plt.subplot(2,2,1) # (number of rows, number of columns, number of plot)
plt.bar(dfDaily[(dfDaily.STATION == "125 ST")].DAY, dfDaily[(dfDaily.STATION == "125 ST")].DailyTotal)
plt.title('125 ST')

plt.subplot(2,2,2,sharey=y1) # (number of rows, number of columns, number of plot)
plt.bar(dfDaily[(dfDaily.STATION == "34 ST-PENN STA")].DAY, dfDaily[(dfDaily.STATION == "34 ST-PENN STA")].DailyTotal)
plt.title('34 ST-PENN STA')

plt.subplot(2,2,3,sharey=y1) # (number of rows, number of columns, number of plot)
plt.bar(dfDaily[(dfDaily.STATION == "14 ST-UNION SQ")].DAY, dfDaily[(dfDaily.STATION == "14 ST-UNION SQ")].DailyTotal)
plt.title('14 ST-UNION SQ')

plt.subplot(2,2,4,sharey=y1) # (number of rows, number of columns, number of plot)
plt.bar(dfDaily[(dfDaily.STATION == "86 ST")].DAY, dfDaily[(dfDaily.STATION == "86 ST")].DailyTotal)
plt.title('86 ST');

In [ ]:
#df.groupby(['STATION', 'DATE', 'TIME'])[['ActualEntry']].sum()
#df.groupby(['STATION', 'DATE'])[['ActualEntry']].sum()
#df.groupby(['DATE','STATION'])[['ActualEntry']].sum().sort_values("ActualEntry", axis = 0, ascending = True)

In [ ]:
dfDaily[(dfDaily.STATION == "59 ST")]
#dfDaily.groupby(['STATION'])[['DailyTotal']].sum()

In [ ]:
#df.to_csv (r'temp.csv', index = False, header=True)